In [1]:
# XGBoost S&P500 closing prices

# CSV from NASDAQ covers 2015-2025

In [2]:
# copy/paste from previous assignments
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import Ridge
#from sklearn.linear_model import Lasso
#from sklearn.ensemble import RandomForestRegressor
#from sklearn import tree

# import graphviz
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import BaggingRegressor
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.ensemble import GradientBoostingRegressor
#from sklearn.ensemble import BaggingClassifier
#from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import GradientBoostingClassifier

# xgboost linear regression
from xgboost import XGBRegressor 

from sklearn.metrics import mean_absolute_error, mean_squared_error


pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML

In [3]:
data = pd.read_csv('/home/manu/Dropbox/SU/1-Spring-25/CIS662-ML/Project/SPX-2015-2025.csv')
data.head()
data.columns

,Date,Close/Last,Open,High,Low
0,03/28/2025,"5,580.94","5,679.20","5,685.89","5,572.42"
1,03/27/2025,"5,693.31","5,695.64","5,732.28","5,670.94"
2,03/26/2025,"5,712.20","5,771.66","5,783.62","5,694.41"
3,03/25/2025,"5,776.65","5,775.96","5,786.95","5,760.42"
4,03/24/2025,"5,767.57","5,718.08","5,775.14","5,718.08"


Index(['Date', 'Close/Last', 'Open', 'High', 'Low'], dtype='object')

In [4]:
data.dtypes

Date           object
Close/Last    float64
Open          float64
High          float64
Low           float64
dtype: object

In [5]:
# date column into python datetime
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values(by='Date')

In [6]:
lag = 5 # 5 trading days in a week usually
for i in range(1, lag + 1):
    data[f'Close_Lag_{i}'] = data['Close/Last'].shift(i)


data['Target'] = data['Close/Last'].shift(-1)
data = data.dropna()

# Features are Close/Last, Open, High, Low,
X = data.drop(columns=['Date', 'Target'])
Y = data['Target']

data.head()
data.columns

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

,Date,Close/Last,Open,High,Low,Close_Lag_1,Close_Lag_2,Close_Lag_3,Close_Lag_4,Close_Lag_5,Target
2516,2015-04-06,"2,080.62","2,064.87","2,086.99","2,056.52","2,066.96","2,066.96","2,059.69","2,067.89","2,086.24","2,076.33"
2515,2015-04-07,"2,076.33","2,080.79","2,089.81","2,076.10","2,080.62","2,066.96","2,066.96","2,059.69","2,067.89","2,081.90"
2514,2015-04-08,"2,081.90","2,076.94","2,086.69","2,073.30","2,076.33","2,080.62","2,066.96","2,066.96","2,059.69","2,091.18"
2513,2015-04-09,"2,091.18","2,081.29","2,093.31","2,074.29","2,081.90","2,076.33","2,080.62","2,066.96","2,066.96","2,102.06"
2512,2015-04-10,"2,102.06","2,091.51","2,102.61","2,091.51","2,091.18","2,081.90","2,076.33","2,080.62","2,066.96","2,092.43"


Index(['Date', 'Close/Last', 'Open', 'High', 'Low', 'Close_Lag_1',
       'Close_Lag_2', 'Close_Lag_3', 'Close_Lag_4', 'Close_Lag_5', 'Target'],
      dtype='object')

In [7]:
model = XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train, Y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [8]:
# what were the results
y_pred = model.predict(X_test)

# Mean Absolute Error
mae = mean_absolute_error(Y_test, y_pred)
mse = mean_squared_error(Y_test, y_pred)
rmse = np.sqrt(mse)

print(f'Errors\nMAE: {mae}, MSE: {mse}, RMSE: {rmse}')

Errors
MAE: 480.19691679454985, MSE: 444536.4546121724, RMSE: 666.735670721293
